In [1]:
from selenium.webdriver.common.by import By
import selenium as sel
import pandas as pd
import time
import datetime

import os as os
import csv

In [30]:

def existedb(url: str):
    """Funcion que verifica si una url existe en la base de datos
    Args:
        url (str): url del articulo a verificar

    Returns:
        Bool: False si la url existe, True si no existe
    """
    
    db = pd.read_csv("../data/raw/semana.csv",encoding='utf8')
    return True if (db["URL"].eq(url)).any() else False
    # with open('../data/raw/articulos.csv', encoding="utf8") as f:
    #     csvreader = csv.reader(f, delimiter=",")
    #     for row in csvreader:
    #         if url in row[5]:
    #              return False
    #     return True


In [12]:
def obtener_tags(driver: sel.webdriver.Edge):
    """Funcion que obtiene los tags del articulo

    Args:
        driver (sel.webdriver.Edge): driver de selenium

    Returns:
        str[]: Lista de strings/tags
    """
    tags = []
    try :
        secTags = driver.find_element(By.XPATH,'.//div[contains(@class,"tags-list")]')
    except:
        tags = 'SIN TAGS'
    else:
        units = secTags.find_elements(By.XPATH,'.//span')
        for i in units:
            tags.append(i.text)
                  
    return tags
    

In [2]:
def obtener_contenido(driver: sel.webdriver.Edge):
    """Funcion que itera sobre todos los parrafos del articulo y los extrae.

    Args:
        driver (sel.webdriver.Edge): driver de selenium

    Returns:
        str: devuelve el contenido del articulo
    """
    contenido = ''
    try :
        html = driver.find_element(By.XPATH,'.//article[contains(@class,"paywall")]')
        parrafos = html.find_elements(By.XPATH,'.//p')
    except:
        parrafos = 'SIN PARRAFOS'
    else:
        for i in parrafos:
            contenido += i.text
       
    return contenido

In [9]:
def obtener_tema(driver: sel.webdriver.Edge):
    """Funcion que itera sobre todos los parrafos del articulo y los extrae.

    Args:
        driver (sel.webdriver.Edge): driver de selenium

    Returns:
        str: devuelve el contenido del articulo
    """
    tema = ''
    try :
        tema = driver.find_element(By.XPATH,'.//h3').text
    except:
        tema = 'SIN TEMA'
       
    return tema

In [13]:
#Empresa con la cual vamos a extraer los articulos
# TODO Tener una lista que itere por todos lo clientes
empresa = input("Digite la empresa a extraer: ")

In [14]:
#cerar driver... MODIFICAR DEPENDIENDO DEL NAVEGADOR
driver = sel.webdriver.Edge()
driver.get(f'https://www.semana.com/buscador/?query={empresa}')
time.sleep(2)

In [15]:
# Extrae la lista de todos los articulos de la pagina
articulos = driver.find_elements(By.XPATH,'.//div[contains(@class,"queryly_item_row")]')

In [16]:
# Itera por cada articulo y extrae la informacion (EN CASO DE QUE NO EXISTA ARCHIVO DE ALMACENAMIENTO ANTERIOR)
titulares = []
for art in articulos:
    url = art.find_element(By.XPATH,'.//a').get_attribute('href')
    fechaP = art.find_element(By.XPATH,'.//div[contains(@style,"margin-bottom:0px;color:#555;font-size:12px;")]').text
    resumen = art.find_element(By.XPATH,'.//div[contains(@class,"queryly_item_description")]').text
    titulo = art.find_element(By.XPATH,'.//div[contains(@class,"queryly_item_title")]').text
    titulares.append({'Fecha Extraccion':datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
                    'Titulo':titulo,
                    'Fecha Publicacion':fechaP,
                    'Resumen':resumen,
                    'URL':url,
                    'Empresa':empresa})
        
    

In [20]:
# busca los autores de cada articulo y las almacena en la lista de titulares
for tit in titulares:
    
    driver.get(tit['URL'])

    # agregar contenido al dict de titulares
    tit['Contenido'] = obtener_contenido(driver)

    # agregar tags al dict de titulares
    tit['Tags'] = obtener_tags(driver)
    
    # agregar contenido al dict de titulares
    tit['Tema'] = obtener_tema(driver)
      

    # se podria agregar un if resumen vacio, llamar a resumen. (para las 3 noticias principales)

In [22]:
print(titulares[1])

{'Fecha Extraccion': '2022-09-05 12:01:39', 'Titulo': 'Moody’s bajó la calificación crediticia de Ecopetrol: el riesgo de refinanciamiento es elevado', 'Fecha Publicacion': 'mié, 24 ago 2022', 'Resumen': 'La firma calificadora de riesgos Moody’s Investors Service anunció este miércoles -24 de agosto- que mantuvo las calificaciones senior quirografarias de Baa3 de Ecopetrol S.A. ...', 'URL': 'https://www.semana.com/economia/inversionistas/articulo/moodys-bajo-la-calificacion-crediticia-de-ecopetrol-el-riesgo-de-refinanciamiento-es-elevado/202241/', 'Empresa': 'Ecopetrol', 'Contenido': 'La firma calificadora de riesgos Moody’s Investors Service anunció este miércoles -24 de agosto- que mantuvo las calificaciones senior quirografarias de Baa3 de Ecopetrol S.A. y que rebajó la Evaluación crediticia de referencia (BCA) de Ecopetrol a ba3 de ba1 debido al elevado riesgo de refinanciamiento de la compañía.Esto, según el informe presentado por esta compañía, es el resultado de una política fin

In [52]:
columnas = ['Fecha Extraccion','Titulo', 'Fecha Publicacion','Tema','Resumen','URL','Imagen','Empresa','Autor','Contenido','RelNewsUrls']

In [53]:
# Para agregar titulares a un archivo.
with open('../data/raw/noticias.csv', 'a', newline='') as csv_file:
    dict_object = csv.DictWriter(csv_file, fieldnames=columnas) 
  
    dict_object.writerows(titulares)

In [41]:
#Para crear un nuevo archivo con los titulares
pd.json_normalize(titulares).to_csv(f'../data/raw/noticias.csv',index=False, encoding='utf8')